In [1]:
import pandas as pd
import subprocess
import ast
import re

## Function to extract the outcome

In [23]:
def extract_case_outcome(text, start, end, model="llama3"):
    """
    Extract the outcome of a legal case in a single word.

    Parameters
    ----------
    text : str
        The court text from which to extract the outcome.
    model : str, optional
        The language model to use (default is "llama3").

    Returns
    -------
    str
        A single word summarizing the case outcome (e.g., "allowed", "dismissed", "granted").
        Returns "unknown" if no outcome is identified.
    """
    lines = text.splitlines()
    relevant_text = "\n".join(lines[start:end])
    prompt = f"""
You are a legal assistant. Your task is to determine the outcome of the court case based on the provided excerpt.

Instructions:
- Read the text carefully and identify the final decision or outcome.
- Return only one lowercase word that best summarizes the outcome (e.g., "allowed", "dismissed", etc).
- If the outcome is unclear or not mentioned, return "unknown".

Court Text:
{relevant_text}

Output:
"""

    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    return result.stdout.decode().strip().lower()

def extract_case_outcomes_from_dataframe(df, start, end, text_column="unofficial_text", model="llama3"):
    """
    Extract the case outcome (single word) from a DataFrame containing court texts.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing court text data.
    text_column : str, optional
        The column name in `df` that contains the court text (default is "unofficial_text").
    model : str, optional
        The language model to use (default is "llama3").

    Returns
    -------
    pandas.DataFrame
        A new DataFrame with an additional 'outcome' column containing the extracted outcome word.
    """
    df = df.copy()
    df["outcome"] = None

    for idx, row in df.iterrows():
        full_text = row[text_column]
        try:
            outcome = extract_case_outcome(full_text, start, end, model=model)
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            outcome = "unknown"

        df.at[idx, "outcome"] = outcome
        print(f"Row {idx} processed with outcome: {outcome}")
    
    return df

## SCC Data

In [20]:
SCC = pd.read_csv("../data/processed/SCC_judges.csv")
SCC

,Unnamed: 0,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges
0,0,2014 SCC 37,SCC,2014,en,2014-05-14,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Citizenship and Immigration) v. Harkat...,"['security', 'human_rights', 'serious_criminal...","['McLachlin', 'LeBel', 'Abella', 'Rothstein', ..."
1,1,2014 SCC 68,SCC,2014,en,2014-10-30,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Febles v. Canada (Citizenship and Immigration)...,['refugee'],"['McLachlin', 'LeBel', 'Abella', 'Rothstein', ..."
2,2,2015 SCC 58,SCC,2015,en,2015-11-27,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,B010 v. Canada (Citizenship and Immigration)\n...,['refugee'],"['McLachlin', 'Abella', 'Rothstein', 'Moldaver..."
3,3,2015 SCC 61,SCC,2015,en,2015-12-10,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee'],"['McLachlin', 'Abella', 'Cromwell', 'Moldaver'..."
4,4,2019 SCC 65,SCC,2019,en,2019-12-19,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Minister of Citizenship and Immigratio...,"['security', 'human_rights', 'serious_criminal...","['Wagner', 'Abella', 'Moldaver', 'Karakatsanis..."
5,5,2023 SCC 17,SCC,2023,en,2023-06-16,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canadian Council for Refugees v. Canada (Citiz...,['refugee'],"['Wagner', 'Karakatsanis', 'Côte', 'Brown', 'R..."
6,6,2023 SCC 21,SCC,2023,en,2023-09-27,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Mason v. Canada (Citizenship and Immigration)\...,['refugee'],[]


In [24]:
SCC_outcome = extract_case_outcomes_from_dataframe(SCC, -30, -10)
SCC_outcome

Row 0 processed with outcome: allowed
Row 1 processed with outcome: allowed
Row 2 processed with outcome: allowed
Row 3 processed with outcome: dismissed
Row 4 processed with outcome: allowed
Row 5 processed with outcome: allowed
Row 6 processed with outcome: allowed


,Unnamed: 0,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges,outcome
0,0,2014 SCC 37,SCC,2014,en,2014-05-14,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Citizenship and Immigration) v. Harkat...,"['security', 'human_rights', 'serious_criminal...","['McLachlin', 'LeBel', 'Abella', 'Rothstein', ...",allowed
1,1,2014 SCC 68,SCC,2014,en,2014-10-30,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Febles v. Canada (Citizenship and Immigration)...,['refugee'],"['McLachlin', 'LeBel', 'Abella', 'Rothstein', ...",allowed
2,2,2015 SCC 58,SCC,2015,en,2015-11-27,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,B010 v. Canada (Citizenship and Immigration)\n...,['refugee'],"['McLachlin', 'Abella', 'Rothstein', 'Moldaver...",allowed
3,3,2015 SCC 61,SCC,2015,en,2015-12-10,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee'],"['McLachlin', 'Abella', 'Cromwell', 'Moldaver'...",dismissed
4,4,2019 SCC 65,SCC,2019,en,2019-12-19,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canada (Minister of Citizenship and Immigratio...,"['security', 'human_rights', 'serious_criminal...","['Wagner', 'Abella', 'Moldaver', 'Karakatsanis...",allowed
5,5,2023 SCC 17,SCC,2023,en,2023-06-16,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Canadian Council for Refugees v. Canada (Citiz...,['refugee'],"['Wagner', 'Karakatsanis', 'Côte', 'Brown', 'R...",allowed
6,6,2023 SCC 21,SCC,2023,en,2023-09-27,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,Mason v. Canada (Citizenship and Immigration)\...,['refugee'],[],allowed


In [28]:
SCC_final = SCC_outcome.drop('Unnamed: 0', axis=1)
SCC_final.to_csv("../data/processed/SCC_outcome.csv")

## FCA

In [29]:
FCA = pd.read_csv("../data/processed/FCA_city.csv")
FCA

,Unnamed: 0.1,Unnamed: 0,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges,locations
0,0,0,2014 FCA 113,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee'],"['BLAIS', 'SHARLOW', 'STRA TAS']",Toronto
1,1,1,2014 FCA 114,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lemus v. Canada (Citizenship and Immigration)\...,['refugee'],"['BLAIS', 'SHARLOW', 'STRA TAS']",Toronto
2,2,2,2014 FCA 126,FCA,2014,en,2014-05-14,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kinsel v. Canada (Citizenship and Immigration)...,"['security', 'human_rights', 'serious_criminal...","['Dawson', 'Trudel', 'Near']",Vancouver
3,3,3,2014 FCA 157,FCA,2014,en,2014-06-10,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Sanchez v. Canada (Citizenship and Immigration...,"['serious_criminality', 'criminality']","['Nadon', 'Stratas', 'Scott']",Vancouver
4,4,4,2014 FCA 160,FCA,2014,en,2014-06-12,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citizenship and Immigration) v. Savin\...,['human_rights'],"['Gauthier', 'Mainville', 'Boivin']",Montréal
...,...,...,...,...,...,...,...,...,...,...,...,...
94,94,94,2023 FCA 36,FCA,2023,en,2023-02-16,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lukács v. Canada (Citizenship and Immigration)...,['refugee'],"['PELLETIER', 'WOODS', 'ROUSSEL']",Ottawa
95,95,95,2024 FCA 165,FCA,2024,en,2024-10-08,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Nanhar v. Canada (Citizenship and Immigration)...,['refugee'],"['Webb', 'Mactavish', 'Walker']",Ottawa
96,96,96,2024 FCA 174,FCA,2024,en,2024-10-24,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Li v. Canada (Citizenship and Immigration)\nCo...,"['human_rights', 'serious_criminality', 'crimi...","['Stratas', 'Boivin', 'Heckman']",Ottawa
97,97,97,2020 CAF 126,FCA,2020,fr,2020-07-28,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citoyenneté et Immigration) c. Solmaz\...,['refugee'],"['Boivin', 'De Montigny', 'Leblanc']",Ottawa


In [30]:
FCA_outcome = extract_case_outcomes_from_dataframe(FCA, -50, -25)
FCA_outcome

Row 0 processed with outcome: dismissed
Row 1 processed with outcome: allowed
Row 2 processed with outcome: dismissed
Row 3 processed with outcome: dismissed
Row 4 processed with outcome: dismissed
Row 5 processed with outcome: dismissed
Row 6 processed with outcome: dismissed
Row 7 processed with outcome: dismissed
Row 8 processed with outcome: unknown
Row 9 processed with outcome: unknown
Row 10 processed with outcome: dismissed
Row 11 processed with outcome: allowed
Row 12 processed with outcome: dismissed
Row 13 processed with outcome: dismissed
Row 14 processed with outcome: dismissed
Row 15 processed with outcome: dismissed
Row 16 processed with outcome: unknown
Row 17 processed with outcome: unknown
Row 18 processed with outcome: dismissed
Row 19 processed with outcome: dismissed
Row 20 processed with outcome: dismissed
Row 21 processed with outcome: dismissed
Row 22 processed with outcome: dismissed
Row 23 processed with outcome: allowed
Row 24 processed with outcome: allowed
R

,Unnamed: 0.1,Unnamed: 0,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges,locations,outcome
0,0,0,2014 FCA 113,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee'],"['BLAIS', 'SHARLOW', 'STRA TAS']",Toronto,dismissed
1,1,1,2014 FCA 114,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lemus v. Canada (Citizenship and Immigration)\...,['refugee'],"['BLAIS', 'SHARLOW', 'STRA TAS']",Toronto,allowed
2,2,2,2014 FCA 126,FCA,2014,en,2014-05-14,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kinsel v. Canada (Citizenship and Immigration)...,"['security', 'human_rights', 'serious_criminal...","['Dawson', 'Trudel', 'Near']",Vancouver,dismissed
3,3,3,2014 FCA 157,FCA,2014,en,2014-06-10,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Sanchez v. Canada (Citizenship and Immigration...,"['serious_criminality', 'criminality']","['Nadon', 'Stratas', 'Scott']",Vancouver,dismissed
4,4,4,2014 FCA 160,FCA,2014,en,2014-06-12,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citizenship and Immigration) v. Savin\...,['human_rights'],"['Gauthier', 'Mainville', 'Boivin']",Montréal,dismissed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,94,94,2023 FCA 36,FCA,2023,en,2023-02-16,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lukács v. Canada (Citizenship and Immigration)...,['refugee'],"['PELLETIER', 'WOODS', 'ROUSSEL']",Ottawa,dismissed
95,95,95,2024 FCA 165,FCA,2024,en,2024-10-08,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Nanhar v. Canada (Citizenship and Immigration)...,['refugee'],"['Webb', 'Mactavish', 'Walker']",Ottawa,dismissed
96,96,96,2024 FCA 174,FCA,2024,en,2024-10-24,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Li v. Canada (Citizenship and Immigration)\nCo...,"['human_rights', 'serious_criminality', 'crimi...","['Stratas', 'Boivin', 'Heckman']",Ottawa,dismissed
97,97,97,2020 CAF 126,FCA,2020,fr,2020-07-28,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citoyenneté et Immigration) c. Solmaz\...,['refugee'],"['Boivin', 'De Montigny', 'Leblanc']",Ottawa,allowed


In [31]:
FCA_final = FCA_outcome.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
FCA_final

,citation,dataset,year,language,document_date,source_url,unofficial_text,inadmissibility_reason,judges,locations,outcome
0,2014 FCA 113,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kanthasamy v. Canada (Citizenship and Immigrat...,['refugee'],"['BLAIS', 'SHARLOW', 'STRA TAS']",Toronto,dismissed
1,2014 FCA 114,FCA,2014,en,2014-05-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lemus v. Canada (Citizenship and Immigration)\...,['refugee'],"['BLAIS', 'SHARLOW', 'STRA TAS']",Toronto,allowed
2,2014 FCA 126,FCA,2014,en,2014-05-14,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Kinsel v. Canada (Citizenship and Immigration)...,"['security', 'human_rights', 'serious_criminal...","['Dawson', 'Trudel', 'Near']",Vancouver,dismissed
3,2014 FCA 157,FCA,2014,en,2014-06-10,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Sanchez v. Canada (Citizenship and Immigration...,"['serious_criminality', 'criminality']","['Nadon', 'Stratas', 'Scott']",Vancouver,dismissed
4,2014 FCA 160,FCA,2014,en,2014-06-12,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citizenship and Immigration) v. Savin\...,['human_rights'],"['Gauthier', 'Mainville', 'Boivin']",Montréal,dismissed
...,...,...,...,...,...,...,...,...,...,...,...
94,2023 FCA 36,FCA,2023,en,2023-02-16,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Lukács v. Canada (Citizenship and Immigration)...,['refugee'],"['PELLETIER', 'WOODS', 'ROUSSEL']",Ottawa,dismissed
95,2024 FCA 165,FCA,2024,en,2024-10-08,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Nanhar v. Canada (Citizenship and Immigration)...,['refugee'],"['Webb', 'Mactavish', 'Walker']",Ottawa,dismissed
96,2024 FCA 174,FCA,2024,en,2024-10-24,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Li v. Canada (Citizenship and Immigration)\nCo...,"['human_rights', 'serious_criminality', 'crimi...","['Stratas', 'Boivin', 'Heckman']",Ottawa,dismissed
97,2020 CAF 126,FCA,2020,fr,2020-07-28,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,Canada (Citoyenneté et Immigration) c. Solmaz\...,['refugee'],"['Boivin', 'De Montigny', 'Leblanc']",Ottawa,allowed


In [32]:
FCA_final['outcome'].value_counts()

outcome
dismissed    58
allowed      30
unknown      10
quashed       1
Name: count, dtype: int64

In [33]:
FCA_final.to_csv("../data/processed/FCA_outcome.csv")